이 계층들을 조합하여 손글씨 숫자를 인식하는 CNN을 조립해보자. 여기에서는 다음과 같은 CNN을 구현한다.

<img src=images/7_23.png height=100px width=500px>

이 그림의 CNN 네트워크는 "Convolution-ReLU-Pooling-Affine-ReLU-Affine-Softmax"순으로 흐른다. 이를 SimpleConvNet이라는 이름의 클래스로 구현하자.

우선 SimpleConvNet의 초기화 (init)을 살펴보자. 초기화 때는 다음 인수들을 받는다.

- input_dim - 입력 데이터(채널 수, 높이, 너비)의 차원
- conv_param - 합성곱 계층의 하이퍼파라미터(딕셔너리). 딕셔너리의 키는 다음과 같다.
    - filter_num - 필터수
    - filter_size - 필터 사이즈
    - stride - 스트라이드
    - pad - 패딩
- hidden_size - 은닉층(완전연결)의 뉴런 수
- output_size - 출력층(완전연결)의 뉴런 수
- weight_init_std - 초기화 때의 가중치 표준편차

여기에서 합성곱 계층의 하이퍼파라미터는 딕셔너리 형태로 주어진다. (conv-param). 이것은 필요한 하이퍼파라미터의 값이 예컨대 {"filter_num": 30, "filter_size": 5, "pad": 0, "stride": 1}처럼 저장된다는 뜻이다. 

In [1]:
import os
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.chdir("/content/drive/My Drive/Colab Notebooks/Deep_Learning")

Mounted at /content/drive


In [2]:
import numpy as np
from collections import OrderedDict
from Layers import *

class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28),
                conv_param={"filter_num": 30, "filter_size": 5, "pad": 0, "stride": 1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        ## part 1
        filter_num = conv_param["filter_num"]
        filter_size = conv_param["filter_size"]
        filter_pad = conv_param["pad"]
        filter_stride = conv_param["stride"]
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad)/filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        ## part 2
        self.params = {}
        self.params["W1"] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params["b1"] = np.zeros(filter_num)
        self.params["W2"] = weight_init_std * \
                            np.random.randn(filter_num, pool_output_size, filter_size, filter_size)
        self.params["b2"] = np.zeros(hidden_size)
        self.params["W3"] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params["b3"] = np.zeros(output_size)
        
        ## part 3
        self.layers = OrderedDict()
        self.layers["Conv1"] = Convolution(self.params["W1"],
                                          self.params["b1"],
                                          conv_param["stride"],
                                          conv_param["pad"])
        self.layers["ReLU1"] = ReLULayer()
        self.layers["Pool1"] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers["Affine1"] = Affine(self.params["W2"], self.params["b2"])
        self.layers["ReLU2"] = ReLULayer()
        self.layers["Affine2"] = Affine(self.params["W3"], self.params["b3"])
        
        self.last_layer = SoftmaxLossLayer()
        
        ## part 4
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        loss = self.last_layer.forward(y, t)
        
        return loss
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
            
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(y, axis=1)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)
            
        return acc / x.shape[0]
        
        ## part 5
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads["W1"], grads["b1"] = self.layers["Conv1"].dW, self.layers["Conv1"].db
        grads["W2"], grads["b2"] = self.layers["Affine1"].dW, self.layers["Affine1"].db
        grads["W3"], grads["b3"] = self.layers["Affine2"].dW, self.layers["Affine2"].db
        
        return grads

##### part 1

여기에서는 초기화 인수로 주어진 합성곱 계층의 하이퍼파라미터를 딕셔너리에서 꺼낸다(나중에 쓰기 쉽도록). 그리고 합성곱 계층의 출력 크기를 계산한다.

##### part 2

학습에 필요한 매개변수는 1번째 층의 합성곱 계층과 나머지 두 완전연결 계층의 가중치와 편향이다. 이 매개변수들을 인스턴스 변수 params 딕셔너리에 저장한다. 1번째 층의 합성곱 계층의 가중치를 W1, 편향을 b1이라는 키로 저장한다. 마찬가지로 2번째 층의 완전연결 계층의 가중치와 편향을 W2와 b2, 마지막 3번째 층의 완전연결 계층의 가중치와 편향을 W3와 b3라는 키로 각각 저장한다.

##### part 3

순서가 있는 $딕셔너리^{OrderedDict}$인 layers에 계층들을 차례로 추가한다. 마지막 SoftmaxWithLoss 계층만큼은 last_layer라는 별고 변수에 저장해둔다.

##### part 4

이상이 SimpleConvNet의 초기화이다. 이렇게 초기화를 마친 다음에는 추론을 수행하는 predict 메소드와 손실 함수의 값을 구하는 loss 메소드를 다음과 같이 구현할 수 있다.

##### summarize
이 코드에서 인수 x는 입력 데이터, t는 정답 레이블이다. 추론을 수행하는 predict 메소드는 초기화 때 layers에 추가한 계층을 맨 앞에서부터 차례로 forward 메소드를 호출하며 그 결과를 다음 계층에 전달한다. 손실 함수를 구하는 loss 메소드는 predict 메소드의 결과를 인수로 마지막 층의 forward 메소드를 호출한다. 즉, 첫 계층부터 마지막 계층까지 forward를 처리한다.

##### part 5

매개변수의 기울기는 오차역전파법으로 구한다. 이 과정은 순전파와 역전파를 반복한다. 지금까지 각 계층의 순전파와 역전파 기능을 제대로 구현했다면, 여기에서는 단지 그것들을 적절한 순서로 호출만 된다. 마지막으로 grads라는 딕셔너리 변수에 각 가중치 매개변수 의 기울기를 저장한다. 이상이 SimpleConvNet의 구현이다. 

이제 이 SimpleConvNet으로 MNIST 데이터셋을 학습해볼 차례이다.

In [3]:
import sys, os
sys.path.append(os.pardir)
from simple_convnet import SimpleConvNet
from optimizer import SGD
from trainer import Trainer
from source.dataset.mnist import load_mnist
import matplotlib.pyplot as plt

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False, one_hot_label=True)

x_train, t_train = x_train[:5000], t_train[:5000]
x_test, t_test = x_test[:1000], t_test[:1000]

network = SimpleConvNet(input_dim=(1,28,28),
                        conv_param={"filter_num": 30, "filter_size": 5, "pad": 0, "stride": 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)

trainer = Trainer(network, x_train, t_train, x_test, t_test)

trainer.train()

KeyboardInterrupt: ignored